# Function Calling
- Function Calling은 OpenAI API에서 특정 함수 정의(JSON schema 기반)를 주고, 모델이 해당 함수를 호출하는 형식의 응답을 생성하게 하는 기능입니다.
- 챗봇, 에이전트(Agent), 도구(Tool) 사용 등 다양한 자동화 및 플러그인 연동에 활용됩니다.

- 지원 모델 gpt-3-turbo, gpt-4, gpt-4-turbo  (2025.1 현재)

- 모델이 우리의 코드를 호출하도록 해서 우리의 함수들을 모델이 호출할수 있도록 하거나
- 모델이 우리가 원하는 특정 모양과 형식의 output 을 갖도록 강제할수 있다.

- https://platform.openai.com/docs/guides/function-calling?api-mode=chat





In [1]:
import os 

from dotenv import load_dotenv
load_dotenv()

print(f'OPENAI_API_KEY={os.getenv("OPENAI_API_KEY")[:20]}...')

OPENAI_API_KEY=sk-proj-iKU13YeoxNgF...


In [2]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

In [3]:
llm = ChatOpenAI(
    temperature=0.1,
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")  # Who 맞다.

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

print(response.content)

I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Rome.


In [4]:
def get_weather(lon, lat):
    print(f'🟡call an api...lon:{lon}, lat:{lat}')    # 가령 이 함수는 실시간 날씨정보 API를 사용하여 가져온다 치자.

# Json scheme 를 활용한 함수의 스키마 정의

In [5]:
function_schema = {
    "name": "get_weather",  # 함수의 이름
    # description : 함수가 무슨 일을 하는지 기술
    # ↓'위도와 경도를 받아서 특정장소의 날씨정보를 가져오는 함수"
    "description": "function that takes longitude and latitude to find the weather of a place",    

    # 파라미터 기술
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {"type": "string","description": "The longitude coordinate",},
            "lat": {"type": "string","description": "The latitude coordinate",},
        },
    },
    # 필수 사항 기술
    "required": ["lon", "lat"],
}

In [6]:
llm = ChatOpenAI(
    temperature=0.1,
).bind(  # <- 여기에 .ChatOpenAI 에게 전달할 인수를 추가
    functions=[
        function_schema,
    ],
    # ↓기본적으로 모델이 특정 함수를 사용하도록 '강제'하거나, 
    #   모델이 함수를 사용하도록 하거나 그냥 답변을 할 수 있도록 모델 스스로 선택하게 할수 있다.

    # 모델을 강제로 함수를 사용하도록 하려면! 아래와 같이 해야 한다
    # function_call={  
    #     "name": "get_weather",  # <-- get_weather 함수를 반드시 사용하라!        
    # },

    # 만약 AI의 선택권을 주어서 AI가 필요에 따라 선택하여 사용하도록 하려면. 아래와 같이 한다.
    function_call="auto"
    
)

In [7]:
chain = prompt | llm
response = chain.invoke({"city": "rome"})

response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 74, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CfNyvxCMeuPbGBOslbpnsFYUcf9pG', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--aec4d43b-281d-43d0-8b88-c2a5947aff69-0', usage_metadata={'input_tokens': 74, 'output_tokens': 24, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# content= 에는 아무 답변이 없다??

"""
AIMessage(content='', 
    additional_kwargs={
        'function_call': {
            'arguments': '{"lon":"12.4964","lat":"41.9028"}', 
            'name': 'get_weather'}, 
            'refusal': None
        }, 
...

"""
None



In [8]:
response.additional_kwargs

{'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}',
  'name': 'get_weather'},
 'refusal': None}

In [10]:
response.additional_kwargs['function_call']

{'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}

In [11]:
response.additional_kwargs['function_call']['arguments']

'{"lon":"12.4964","lat":"41.9028"}'

In [12]:
import json
r = json.loads(response.additional_kwargs['function_call']['arguments'])
r

{'lon': '12.4964', 'lat': '41.9028'}

In [13]:
r['lon'], r['lat']

('12.4964', '41.9028')

In [14]:
get_weather(r['lon'], r['lat'])

🟡call an api...lon:12.4964, lat:41.9028


In [15]:
get_weather(**r)

🟡call an api...lon:12.4964, lat:41.9028


# 퀴즈 생성하기 스키마

In [16]:
# ↓ 우라기 원하는 답변의 스키마는
 
# 바로 이 형태를 정의하는 거다. 
# {
#   "questions":[
#      0:{
#        "question":"What … in the story?"
#        "answers":[
#           0:{
#             "answer":"John"
#             "correct":false
#           }
#           ... 
#        ]
#      }
#      ...
# }     

In [17]:
function_schema = {
    # '퀴즈 생성하기' 스키마 
    "name": "create_quiz",
    # ↓ 완전히 지어낸 함수다!
    "description": "function that takes a list of questions and answers and returns a quiz", 

    "parameters": {
        # 바로 여기에 우리가 받길 원하는 포맷을 기술하는 것이다!
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string"
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {"type": "string"},
                                    "correct": {"type": "boolean"},
                                },
                                "required": ["answer", "correct"],
                            },                            
                        },
                    },
                    "required": ["question", "answers"],
                },
            },
        },
    },

    "required": ["questions"],
    
}


In [18]:
llm = ChatOpenAI(
    temperature=0.1,
).bind(
    functions=[
        function_schema  # 여기에는 원하는 만큼의 function 들을 적어 넣을수 있다.
    ],
    function_call={
        "name": "create_quiz",  
    },
)

# prompt 변경
prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

response = response.additional_kwargs['function_call']['arguments']

response


'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1000 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by the ancient Romans?","answers":[{"answer":"Eiffel Tower","correct":false},{"answer":"Colosseum","correct":true},{"answer":"Big Ben","correct":false}]}]}'

In [19]:
from pprint import pprint

pprint(json.loads(response))

{'questions': [{'answers': [{'answer': '753 BC', 'correct': True},
                            {'answer': '476 AD', 'correct': False},
                            {'answer': '1000 AD', 'correct': False}],
                'question': 'What year was Rome founded?'},
               {'answers': [{'answer': 'Julius Caesar', 'correct': False},
                            {'answer': 'Augustus', 'correct': True},
                            {'answer': 'Nero', 'correct': False}],
                'question': 'Who was the first emperor of Rome?'},
               {'answers': [{'answer': 'Eiffel Tower', 'correct': False},
                            {'answer': 'Colosseum', 'correct': True},
                            {'answer': 'Big Ben', 'correct': False}],
                'question': 'What famous structure in Rome was built by the '
                            'ancient Romans?'}]}
